In [1]:
import json
import ray
from ray.rllib.agents.registry import get_agent_class
from ray.tune import run_experiments
from ray.tune.registry import register_env
import numpy as np
from flow.networks.ring import RingNetwork, ADDITIONAL_NET_PARAMS
from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder
from flow.core.params import SumoParams, EnvParams, InitialConfig, NetParams
from flow.core.params import VehicleParams, SumoCarFollowingParams
from flow.controllers import RLController, IDMController, ContinuousRouter
from gym.spaces.box import Box
from gym.spaces import Tuple
from flow.core.params import InFlows
from flow.controllers import SimLaneChangeController
from flow.networks import Network
import os
from flow.controllers.routing_controllers import ConstructionRouter
from flow.core.params import SumoLaneChangeParams

from env_testV1 import myEnv

ADDITIONAL_ENV_PARAMS = {
    "max_accel": 1,
    "max_decel": 1,
}

# time horizon of a single rollout
HORIZON = 1500
# number of rollouts per training iteration
N_ROLLOUTS = 20
# number of parallel workers
N_CPUS = 2

# We place one autonomous vehicle and 22 human-driven vehicles in the network
vehicles = VehicleParams()
vehicles.add("rl",
             acceleration_controller=(RLController, {}),
             lane_change_controller=(SimLaneChangeController, {}),
             routing_controller=(ConstructionRouter, {}),
             car_following_params=SumoCarFollowingParams(
                 speed_mode="obey_safe_speed",  
             ),
             num_vehicles=1)
vehicles.add("human",
             acceleration_controller=(IDMController, {}),
             lane_change_controller=(SimLaneChangeController, {}),
             routing_controller=(ContinuousRouter, {}),
             car_following_params=SumoCarFollowingParams(
                 speed_mode="obey_safe_speed",  
             ),
             lane_change_params = SumoLaneChangeParams(
                 lane_change_mode="aggressive",
                 lc_speed_gain=5.0),
             num_vehicles=200)

# specify the edges vehicles can originate on
initial_config = InitialConfig(
    edges_distribution=["gneE15","gneE13","gneE35"]
)
    
# specify the routes for vehicles in the network
class Network(Network):

    def specify_routes(self, net_params):
        return {"gneE15": ["gneE15","gneE13","gneE4.264","gneE8","gneE9","gneE33","gneE35","gneE15"],
                "gneE13": ["gneE13","gneE4.264","gneE8","gneE9","gneE33","gneE35","gneE15","gneE13"],
                "gneE35": ["gneE35","gneE15","gneE13","gneE4.264","gneE8","gneE9","gneE33","gneE35"],
               "gneE8": ["gneE8","gneE7","gneE6","gneE5","gneE8"]
               }


inflow = InFlows()
inflow.add(veh_type="human",
           edge="gneE4",
           vehs_per_hour=5000,
            depart_lane="random",
            depart_speed="random",
            color="white")
# inflow.add(veh_type="human",
#            edge="gneE15",
#            vehs_per_hour=300,
#             depart_lane="2",
#             depart_speed="random",
#             color="white")
# inflow.add(veh_type="rl",
#            edge="gneE15",
#            vehs_per_hour=200,
#             depart_lane="2",
#             depart_speed="random",
#             color="red")

file_dir = "/home/llavezzo/"
net_params = NetParams(
    template="/mnt/c/Users/llave/Documents/GitHub/flow_osuphysics/lucalavezzo/constructionV3.net.xml"
    #inflows=inflow
)

flow_params = dict(
    # name of the experiment
    exp_tag="construction_traffic",

    # name of the flow environment the experiment is running on
    env_name=myEnv,  # <------ here we replace the environment with our new environment

    # name of the network class the experiment is running on
    network=Network,

    # simulator that is used by the experiment
    simulator='traci',

    # sumo-related parameters (see flow.core.params.SumoParams)
    sim=SumoParams(
        sim_step=0.1,
        render=False,
        restart_instance=True
    ),

    # environment related parameters (see flow.core.params.EnvParams)
    env=EnvParams(
        horizon=HORIZON,
        warmup_steps=750,
        clip_actions=False,
        additional_params={
            "target_velocity": 20,
            "sort_vehicles": False,
            "max_accel": 1,
            "max_decel": 1,
        },
    ),

    # network-related parameters (see flow.core.params.NetParams and the
    # network's documentation or ADDITIONAL_NET_PARAMS component)
    net=net_params,

    # vehicles to be placed in the network at the start of a rollout (see
    # flow.core.params.VehicleParams)
    veh=vehicles,

    # parameters specifying the positioning of vehicles upon initialization/
    # reset (see flow.core.params.InitialConfig)
    initial=initial_config
)


def setup_exps():
    """Return the relevant components of an RLlib experiment.

    Returns
    -------
    str
        name of the training algorithm
    str
        name of the gym environment to be trained
    dict
        training configuration parameters
    """
    alg_run = "PPO"

    agent_cls = get_agent_class(alg_run)
    config = agent_cls._default_config.copy()
    config["num_workers"] = N_CPUS
    config["train_batch_size"] = HORIZON * N_ROLLOUTS
    config["gamma"] = 0.999  # discount rate
    config["model"].update({"fcnet_hiddens": [3, 3]})
    config["use_gae"] = True
    config["lambda"] = 0.97
    config["kl_target"] = 0.02
    config["num_sgd_iter"] = 10
    config['clip_actions'] = False  # FIXME(ev) temporary ray bug
    config["horizon"] = HORIZON

    # save the flow params for replay
    flow_json = json.dumps(
        flow_params, cls=FlowParamsEncoder, sort_keys=True, indent=4)
    config['env_config']['flow_params'] = flow_json
    config['env_config']['run'] = alg_run

    create_env, gym_name = make_create_env(params=flow_params, version=0)

    # Register as rllib env
    register_env(gym_name, create_env)
    return alg_run, gym_name, config


/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.

In [2]:
alg_run, gym_name, config = setup_exps()
ray.init(num_cpus=N_CPUS + 1)
trials = run_experiments({
    flow_params["exp_tag"]: {
        "run": alg_run,
        "env": gym_name,
        "config": {
            **config
        },
        "checkpoint_freq": 20,
        "checkpoint_at_end": True,
        "max_failures": 999,
        "stop": {
            "training_iteration": 200,
        },
    }
})


2020-03-04 14:09:37,043	WARNING services.py:592 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-03-04 14:09:37,046	INFO resource_spec.py:212 -- Starting Ray with 4.83 GiB memory available for workers and up to 2.43 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-03-04 14:09:37,466	WARNING services.py:1080 -- Failed to start the dashboard. The dashboard requires Python 3 as well as 'pip install aiohttp psutil setproctitle grpcio'.


Trial name,status,loc
PPO_myEnv-v0_b1dc02ca,RUNNING,


(pid=130) E0304 14:09:40.033189600     130 socket_utils_common_posix.cc:208] check for SO_REUSEPORT: {"created":"@1583348980.033170100","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","file_line":185,"os_error":"Protocol not available","syscall":"getsockopt(SO_REUSEPORT)"}
(pid=130) E0304 14:09:40.033465400     130 socket_utils_common_posix.cc:313] setsockopt(TCP_USER_TIMEOUT) Protocol not available
(pid=131) E0304 14:09:40.033807700     131 socket_utils_common_posix.cc:208] check for SO_REUSEPORT: {"created":"@1583348980.033782500","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","file_line":185,"os_error":"Protocol not available","syscall":"getsockopt(SO_REUSEPORT)"}
(pid=131) E0304 14:09:40.034138600     131 socket_utils_common_posix.cc:313] setsockopt(TCP_USER_TIMEOUT) Protocol not available
(pid=raylet) E03

(pid=130) /home/llavezzo/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=130)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=130) /home/llavezzo/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=130)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=130) /home/llavezzo/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=130)   _np_qint16 = np.dtype([("qint16", np.int16, 1)])
(pid

(pid=129) Warning: Vehicle 'human_18' performs emergency braking with decel=-21.93 wished=4.50 severity=3.87, time=39.10.
(pid=129) Warning: Vehicle 'human_11' performs emergency braking with decel=-75.64 wished=4.50 severity=15.81, time=40.70.
(pid=129) Warning: Vehicle 'human_7' performs emergency braking with decel=-10.31 wished=4.50 severity=1.29, time=40.80.
(pid=130) Warning: Vehicle 'human_11' performs emergency braking with decel=-74.50 wished=4.50 severity=15.56, time=41.30.
(pid=130) Warning: Vehicle 'human_7' performs emergency braking with decel=-9.56 wished=4.50 severity=1.13, time=41.40.
(pid=129) Warning: Vehicle 'human_11' performs emergency braking with decel=-9.78 wished=4.50 severity=1.17, time=41.80.
(pid=130) Warning: Vehicle 'human_6' performs emergency braking with decel=-37.91 wished=4.50 severity=7.43, time=42.00.
(pid=130) Warning: Vehicle 'human_6' performs emergency braking with decel=-29.55 wished=4.50 severity=5.57, time=42.50.
(pid=129) Warning: Vehicle '

...

(pid=130) Warning: Vehicle 'human_191' performs emergency braking with decel=-51.88 wished=4.50 severity=10.53, time=77.00.
(pid=129) Warning: Vehicle 'human_4' performs emergency braking with decel=-21.97 wished=4.50 severity=3.88, time=69.10.
(pid=130) Warning: Vehicle 'human_193' performs emergency braking with decel=-9.49 wished=4.50 severity=1.11, time=77.80.
(pid=130) Warning: Vehicle 'human_34' performs emergency braking with decel=-22.13 wished=4.50 severity=3.92, time=80.30.
(pid=130) Warning: Vehicle 'human_32' performs emergency braking with decel=-13.07 wished=4.50 severity=1.90, time=80.40.
(pid=129) Warning: Vehicle 'human_2' performs emergency braking with decel=-22.92 wished=4.50 severity=4.09, time=72.00.
(pid=129) Warning: Vehicle 'human_4' performs emergency braking with decel=-9.83 wished=4.50 severity=1.19, time=73.10.
(pid=130) Warning: Vehicle 'human_189' performs emergency braking with decel=-17.66 wished=4.50 severity=2.92, time=83.30.
(pid=129) Warning: Vehicl

(pid=130) Warning: Vehicle 'human_174' performs emergency braking with decel=-18.82 wished=4.50 severity=3.18, time=142.50.
(pid=129) Warning: Vehicle 'human_155' performs emergency braking with decel=-14.38 wished=4.50 severity=2.20, time=133.80.
(pid=130) Warning: Vehicle 'human_133' performs emergency braking with decel=-11.45 wished=4.50 severity=1.55, time=144.70.
(pid=129) Warning: Vehicle 'human_182' performs emergency braking with decel=-19.05 wished=4.50 severity=3.23, time=135.70.
(pid=129) Warning: Vehicle 'human_133' performs emergency braking with decel=-21.60 wished=4.50 severity=3.80, time=138.00.
(pid=130) Warning: Vehicle 'human_170' performs emergency braking with decel=-17.09 wished=4.50 severity=2.80, time=148.30.
(pid=129) Warning: Vehicle 'human_192' performs emergency braking with decel=-15.25 wished=4.50 severity=2.39, time=140.00.
(pid=129) Warning: Vehicle 'human_178' performs emergency braking with decel=-11.87 wished=4.50 severity=1.64, time=141.50.
(pid=130

(pid=130) Warning: Vehicle 'human_136' performs emergency braking with decel=-21.93 wished=4.50 severity=3.87, time=202.10.
(pid=129) Warning: Vehicle 'human_154' performs emergency braking with decel=-13.43 wished=4.50 severity=1.98, time=194.60.
(pid=130) Warning: Vehicle 'human_146' performs emergency braking with decel=-18.95 wished=4.50 severity=3.21, time=204.40.
(pid=129) Warning: Vehicle 'human_131' performs emergency braking with decel=-10.83 wished=4.50 severity=1.41, time=196.40.
(pid=130) Warning: Vehicle 'human_136' performs emergency braking with decel=-18.42 wished=4.50 severity=3.09, time=205.00.
(pid=129) Warning: Vehicle 'human_154' performs emergency braking with decel=-18.55 wished=4.50 severity=3.12, time=197.60.
(pid=130) Warning: Vehicle 'human_77' performs emergency braking with decel=-18.80 wished=4.50 severity=3.18, time=206.70.
(pid=130) Warning: Vehicle 'human_103' performs emergency braking with decel=-50.44 wished=4.50 severity=10.21, time=207.50.
(pid=129

(pid=129) Warning: Vehicle 'human_11' performs emergency braking with decel=-93.69 wished=4.50 severity=19.82, time=38.50.
(pid=129) Warning: Vehicle 'human_10' performs emergency braking with decel=-44.20 wished=4.50 severity=8.82, time=39.20.
(pid=129) Warning: Vehicle 'human_10' performs emergency braking with decel=-49.10 wished=4.50 severity=9.91, time=39.50.
(pid=130) Warning: Vehicle 'human_10' performs emergency braking with decel=-15.67 wished=4.50 severity=2.48, time=47.40.
(pid=130) Warning: Vehicle 'human_4' performs emergency braking with decel=-19.35 wished=4.50 severity=3.30, time=48.20.
(pid=129) Warning: Vehicle 'human_11' performs emergency braking with decel=-23.95 wished=4.50 severity=4.32, time=42.40.
(pid=130) Warning: Vehicle 'human_3' performs emergency braking with decel=-18.84 wished=4.50 severity=3.19, time=50.50.
(pid=130) Warning: Vehicle 'human_10' performs emergency braking with decel=-20.40 wished=4.50 severity=3.53, time=50.70.
(pid=129) Warning: Vehicl

KeyboardInterrupt: 